In [ ]:
!unzip -qq "/content/drive/MyDrive/Sample.zip" -d '/content/sample/'

In [ ]:
import json
import os
import shutil


In [ ]:
def delete_json_files_not_in_png_folder(a_folder, b_folder):
    for root, dirs, files in os.walk(a_folder):
        for a_file_name in files:
            if a_file_name.endswith('.json'):
                a_file_path = os.path.join(root, a_file_name)
                corresponding_png_file_path = os.path.join(b_folder, os.path.relpath(a_file_path, a_folder).replace('.json', '.png'))

                # b폴더에 동일한 경로에 .png 파일이 없으면 .json 파일을 삭제합니다.
                if not os.path.exists(corresponding_png_file_path):
                    print(f"Deleting {a_file_path}")
                    os.remove(a_file_path)

# 사용 예시:
a_folder_path = '/content/sample/Sample/02.라벨링데이터'
b_folder_path = '/content/sample/Sample/01.원천데이터'

delete_json_files_not_in_png_folder(a_folder_path, b_folder_path)



In [ ]:
ufo = {
    'images': {}
}


In [ ]:
def find_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                json_files.append(os.path.join(root, file))
    return json_files

In [ ]:
def coco_to_ufo(input_path,output_path):

    for path in input_path:
        with open(path, 'r') as f:
            file=json.load(f)
            image_name = file['Images']['identifier']+'.png'
            if image_name not in ufo['images']:
                ufo['images'][image_name] = {
                    "paragraphs": {},
                    "words": {},
                    "chars": {},
                    "img_w": file['Images']['width'],
                    "img_h": file['Images']["height"],
                    "tags": ["re-annotated"],
                    "relations": {},
                    "annotation_log": {
                        "worker": "",
                        "timestamp": "",
                        "tool_version": "",
                        "source": None
                        },
                    "license_tag": {
                        "usability": True,
                        "public": True,
                        "commercial": True,
                        "type": None,
                        "holder": "AIHUB"
                        }
                    }
                id = 1
                for bbox in file['bbox']:
                    if len(bbox['x']) == 4:
                        ufo['images'][image_name]['words'][str(id).zfill(4)] = {
                            'transcription':"",
                            'points':[[bbox['x'][0],bbox['y'][0]], [bbox['x'][2],bbox['y'][2]], [bbox['x'][3],bbox['y'][3]] ,[bbox['x'][1],bbox['y'][1]]],
                            'orientation':'Horizontal',
                            "language": None,
                            "tags": ['Auto'],
                            "confidence": None,
                            "illegibility": False
                        }
                    id+=1


    with open(output_path, "w") as f:
        json.dump(ufo, f, indent=4)

In [ ]:
coco_to_ufo(find_json_files('/content/sample/Sample/02.라벨링데이터'),'/content/output.json')

In [ ]:
def move_files_to_data_folder(folder_a):
    # 폴더 A 안의 모든 폴더 목록을 가져옵니다.
    subfolders = [f.path for f in os.scandir(folder_a) if f.is_dir()]

    # "data" 폴더 생성
    data_folder = os.path.join(folder_a, "data")
    os.makedirs(data_folder, exist_ok=True)

    # 각 폴더의 파일을 "data" 폴더로 이동
    for subfolder in subfolders:
        for root, dirs, files in os.walk(subfolder):
            for file_name in files:
                source_path = os.path.join(root, file_name)
                destination_path = os.path.join(data_folder, file_name)

                # 파일을 "data" 폴더로 이동
                shutil.move(source_path, destination_path)
                print(f"Moved: {source_path} -> {destination_path}")

# 사용 예시:
folder_a_path = '/content/sample/Sample/01.원천데이터'
move_files_to_data_folder(folder_a_path)

In [ ]:
# 압축할 폴더 경로
folder_to_compress = '/content/sample/Sample/01.원천데이터/data'

# 압축된 파일 저장 경로 및 이름
zip_file_path = '/content/d.zip'

# 압축 명령 실행
os.system(f"zip -r {zip_file_path} {folder_to_compress}")

0